<h1 style="align: center;font-size: 18pt;">Restraint</h1>

<hr style="height:2.5px">

This tutorial shows the user how to appropriately use the `Restraint class` to building the ensemble by importing the data files that were prepared from the previous tutorial [Preparation](https://biceps.readthedocs.io/en/latest/examples/Tutorials/Prep_Rest_Post_Ana/preparation.html) via the `Preparation class`. **Please note** that in order to compute the relative free energies, one must perform sampling for at least two lambda values. For simplicity, we will construct an ensemble for only a single lambda value.

<hr style="height:2.5px">

In [1]:
import sys, os
import numpy as np
import biceps

BICePs - Bayesian Inference of Conformational Populations, Version 2.0


In [2]:
print(f"Possible input data extensions: {biceps.toolbox.list_possible_extensions()}")

Possible input data extensions: ['H', 'Ca', 'N', 'J', 'noe', 'pf']


In [3]:
####### Data and Output Directories #######
energies = np.loadtxt('cineromycin_B/cineromycinB_QMenergies.dat')*627.509  # convert from hartrees to kcal/mol
energies = energies/0.5959   # convert to reduced free energies F = f/kT
energies -= energies.min()  # set ground state to zero, just in case
states = len(energies)
top = 'cineromycin_B/cineromycinB_pdbs/0.fixed.pdb'

# Point to directory that contains input files 
data = biceps.toolbox.sort_data('cineromycin_B/J_NOE')
print(f"Input data: {biceps.toolbox.list_extensions(data)}")

# Make a new directory if we have to
outdir = 'results'
biceps.toolbox.mkdir(outdir)

Input data: ['J', 'noe']


Another key parameter for BICePs set-up is the type of reference potential for each experimental observables. More information of reference potential can be found [here](https://biceps.readthedocs.io/en/latest/theory.html).

Three reference potentials are supported in BICePs: uniform ('uniform'), exponential ('exp'), Gaussian ('gau').  

As we found in previous research, exponential reference potential is useful in most cases. Some higher level task may require more in reference potential selection (e.g [force field parametrization](https://pubs.acs.org/doi/10.1021/acs.jpcb.7b11871)).

**(Note: It will be helpful to print out what is the order of experimental observables included in BICePs sampling as shown above.)**

The order of the parameters below must follow the order of `biceps.toolbox.list_extensions(data)`

In [6]:
#   ['J', 'noe']
ref=['uniform','exp']
uncern=[[0.05,20.0,1.02],[0.05,5.0,1.02]]
gamma = [0.2,5.0,1.01]
n_lambdas = 2
lambda_values = np.linspace(0.0, 1.0, n_lambdas)

#### Quick note on lambda values:
We need to specify what lambda value(s) we want to use in BICePs samplings. Briefly, lambda values are similar to the parameters used in free energy perturbation (FEP) and has effect on the BICePs score. The lambda values represent how much prior information from computational modeling is included in BICePs sampling (1.0 means all, 0.0 means none). As we explained in [this work](https://pubs.acs.org/doi/10.1021/acs.jpcb.7b11871), one can consider BICePs score as the relative free energy change between different models. More lambda values will increase the samplings for [multistate Bennett acceptance ratio (MBAR)](http://www.alchemistry.org/wiki/Multistate_Bennett_Acceptance_Ratio) predictions in free energy change and populations. However more lambda values also will slow down the whole process of BICePs (as more samplings need to run), so balancing the accuracy and efficiency is important. To successfully finish a BICePs sampling, lambda values of 0.0 and 1.0 are necessary. Based on our experience, the lambda values of 0.0,0.5,1.0 are good enough for BICePs sampling.
**In this tutorial, we will go through a single lambda for simplicity.**

In [8]:
import pickle
for lam in lambda_values:
    print(f"lambda: {lam}")
    ensemble = biceps.Ensemble(lam, energies, top, verbose=False)
    ensemble.initialize_restraints(input_data=data, ref_pot=ref,
            uncern=uncern, gamma=gamma)
    # Save each ensemble as a pickle file
    print(f"Saving ensemble_{lam}.pkl ...")
    with open(outdir+"/ensemble_%s.pkl"%lam,'wb') as file:
        pickle.dump(ensemble, file)

lambda: 0.0
Saving ensemble_0.0.pkl ...
lambda: 1.0
Saving ensemble_1.0.pkl ...


#### Let's take a look at the ensemble (lam=1.0).
The ensemble consists of a list of 2 restraint objects for each state. Here we are showing the first 10 states.

In [10]:
print(ensemble.to_list()[:10])

[[<biceps.Restraint.Restraint_J object at 0x1a25e2be10>, <biceps.Restraint.Restraint_noe object at 0x1a26043438>], [<biceps.Restraint.Restraint_J object at 0x1a2607a0f0>, <biceps.Restraint.Restraint_noe object at 0x1a25ecd400>], [<biceps.Restraint.Restraint_J object at 0x1a2605f080>, <biceps.Restraint.Restraint_noe object at 0x1a261b8588>], [<biceps.Restraint.Restraint_J object at 0x1a26280d68>, <biceps.Restraint.Restraint_noe object at 0x1a25ed7780>], [<biceps.Restraint.Restraint_J object at 0x1a26068780>, <biceps.Restraint.Restraint_noe object at 0x1a25f37080>], [<biceps.Restraint.Restraint_J object at 0x1a25dde4a8>, <biceps.Restraint.Restraint_noe object at 0x1a263bdf28>], [<biceps.Restraint.Restraint_J object at 0x1a25d73cf8>, <biceps.Restraint.Restraint_noe object at 0x1a25d73f28>], [<biceps.Restraint.Restraint_J object at 0x1a262be1d0>, <biceps.Restraint.Restraint_noe object at 0x1a263411d0>], [<biceps.Restraint.Restraint_J object at 0x1a26167400>, <biceps.Restraint.Restraint_noe

### Conclusion###
In this tutorial, we explained how to use [Restraint](https://biceps.readthedocs.io/en/latest/biceps.html#restraint) to construct the ensemble, which we saved as a pickle file. In the next tutorial, [PosteriorSampler](https://biceps.readthedocs.io/en/latest/examples/Tutorials/Prep_Rest_Post_Ana/posteriorsampler.html) we will Sample the posterior distribution by using the `PosteriorSampler` class.

<h6 style="align: justify;font-size: 12pt"># <span style="color:red;">NOTE</span>: The following cell is for pretty notebook rendering</h6>

In [7]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../../theme.css", "r").read()
    return HTML(styles)
css_styling()